# <code>SymPy</code> for Finite Difference schemes
Finite difference schemes are used to estimate higher derivatives using the function values at surrounding points. The function is assumed to admit Taylor series (as is the case with most physical fields) and the surrounding values are expanded in Taylor series and higher derivatives are matched to get the coefficients. This process can be coded with <code>SymPy</code> (or other symbolic manipulation software)

## F.D. Schemes
A typical finite difference scheme for the first derivative looks like this :
\begin{equation}
    \sum_{n=-1}^{+1} l_{i+n} f'(x_{i+n}) = \sum_{n=-2}^{+2} r_{i+n} f(x_{i+n})
\end{equation}
The first derivatives (R.H.S.) and function values (L.H.S.) are related linearly and coefficients $l_{i+n}, r_{i+n}$ are evaluated such that higher derivatives are matched in the scheme. Here, we have 3 $f'$ terms in L.H.S. and 5 $f$ terms in R.H.S.

## Taylor series
We expand all the $f$ terms i.e., function and it's derivative using Taylor series about the point of interest (where we need the derivative). A typical Taylor series expansion of $f(x_{i+1})$ about $x_{i}$ (point of interest) is given below :
\begin{equation}
f(x_{i+1}) = f(x_{i}) + h_r f'(x_{i}) + \dfrac{h_r^2}{2!}f''(x_{i}) + \cdots
\end{equation}
where $h_r=x_{i+1}-x_{i}$

## Example
Let's evaluate the coefficients for an explicit three-point first derivative scheme given as $f'_i = a f_{i-1} + b f_{i} + c f_{i+1}$ using <code>SymPy</code>

In [4]:
## Importing required packages
from sympy import symbols, Function, series, diff, Eq, linsolve, pprint

In [21]:
## Declaring symbols
x, x0, hl, hr = symbols('x x0 hl hr')
a, b, c       = symbols('a b c')
## Declaring function
f             = symbols('f', cls=Function, real=True)

In [35]:
## Taylor series of all terms { f_{i+1}, f_{i-1} } about x0 upto n(=4) terms
fr = series(f(x), x=x, x0=x0, n=4, dir='+').doit().subs(x, x0+hr) # fr (f right) => f_{i+1}
fl = series(f(x), x=x, x0=x0, n=4, dir='-').doit().subs(x, x0-hl) # fr (f left)  => f_{i-1}
print('\n') # empty line just for spacing
pprint(fr)  # pretty print of fr
pprint(fl)  # pretty print of fl



                              2                 3                 
                          2  d              3  d                  
                        hr ⋅────(f(x₀))   hr ⋅────(f(x₀))         
                               2                 3                
            d               dx₀               dx₀            ⎛  4⎞
f(x₀) + hr⋅───(f(x₀)) + ─────────────── + ─────────────── + O⎝hr ⎠
           dx₀                 2                 6                
                              2                 3                 
                          2  d              3  d                  
                        hl ⋅────(f(x₀))   hl ⋅────(f(x₀))         
                               2                 3                
            d               dx₀               dx₀            ⎛  4⎞
f(x₀) - hl⋅───(f(x₀)) + ─────────────── - ─────────────── + O⎝hl ⎠
           dx₀                 2                 6                


In [76]:
## the scheme written as L.H.S. - R.H.S.
stencil = diff(f(x0), x0, n=1) - a*fl - b*f(x0) - c*fr
# expands and removes the last order terms O(h^4)
stencil = stencil.expand().removeO()
print('\n')
pprint(stencil)



        3                   2                                                 
    3  d                2  d                                                  
a⋅hl ⋅────(f(x₀))   a⋅hl ⋅────(f(x₀))                                         
         3                   2                                                
      dx₀                 dx₀                 d                               
───────────────── - ───────────────── + a⋅hl⋅───(f(x₀)) - a⋅f(x₀) - b⋅f(x₀) - 
        6                   2                dx₀                              

        3                   2                                                 
    3  d                2  d                                                  
c⋅hr ⋅────(f(x₀))   c⋅hr ⋅────(f(x₀))                                         
         3                   2                                                
      dx₀                 dx₀                 d                      d        
───────────────── - ───────────────── - c⋅hr⋅───(

In [69]:
# here cn are coefficient of n th derivative in stencil expression where n={0,1,2,3}
c0 = stencil.coeff(diff(f(x0),x0,0)); print(c0)
c1 = stencil.coeff(diff(f(x0),x0,1)); print(c1)
c2 = stencil.coeff(diff(f(x0),x0,2)); print(c2)
c3 = stencil.coeff(diff(f(x0),x0,3)); print(c3)

-a - b - c
a*hl - c*hr + 1
-a*hl**2/2 - c*hr**2/2
a*hl**3/6 - c*hr**3/6


In [71]:
## Solution of linear system using linsolve
# Eq(c0,0) is [c0 == 0] and we solve for a, b and c (3 unknowns) satisfying c0, c1, c2 (3 equations)
sol = linsolve([Eq(c0,0), Eq(c1,0), Eq(c2,0)], (a, b, c))
# We print the solution obtained
pprint(sol)

⎧⎛    -hr       -hl + hr       hl     ⎞⎫
⎨⎜────────────, ────────, ────────────⎟⎬
⎩⎝hl⋅(hl + hr)   hl⋅hr    hr⋅(hl + hr)⎠⎭


In [73]:
print('Leading Order Truncation Error Term is :')
# leading order error is obtained by substituting the solution in stencil expression (used n=4)
pprint(c3.subs([(a,sol.args[0][0]),(b,sol.args[0][1]),(c,sol.args[0][2])]).simplify()*diff(f(x0),x0,3))

Leading Order Truncation Error Term is :
         3         
        d          
-hl⋅hr⋅────(f(x₀)) 
          3        
       dx₀         
───────────────────
         6         


## This concludes the demonstration of <code>SymPy</code> for the example scheme. This can be extended for higher order explicit and implicit schemes seamlessly.